In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import xgboost as xgb
import numpy as np
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.inspection import partial_dependence
import threading
from datetime import datetime
import json
import os
import difflib

class CompressiveStrengthPredictor:
    def __init__(self):
        self.setup_style()
        self.load_data()
        self.train_model()
        self.create_gui()
        
    def setup_style(self):
        self.colors = {
            'primary': '#2E86AB',
            'secondary': '#A23B72',
            'accent': '#F18F01',
            'background': '#F7F9FC',
            'card_bg': '#FFFFFF',
            'success': '#28A745',
            'warning': '#FFC107',
            'danger': '#DC3545',
            'info': '#17A2B8',
            'dark': '#343A40'
        }
        
    def load_data(self):
        try:
            # Load the new dataset
            self.df = pd.read_csv(r'D:\2026 Work\Dr Deeshani\Data 1\Data CS\CS.csv', 
                                 encoding='ISO-8859-1')
            print(f"📋 Original columns: {len(self.df.columns)}")
            
            # Clean column names
            self.df.columns = self.df.columns.str.strip()
            print(f"✅ Cleaned columns: {len(self.df.columns)}")
            print(f"📐 Dataset shape: {self.df.shape}")
            
            # Find target column
            target_col = 'Compressive strength (MPa)'
            if target_col not in self.df.columns:
                matches = difflib.get_close_matches('Compressive strength (MPa)', self.df.columns, n=1)
                if matches:
                    target_col = matches[0]
                    print(f"🎯 Using '{target_col}' as target column")
            
            # Check if target column exists
            if target_col not in self.df.columns:
                print("Available columns:", self.df.columns.tolist())
                raise ValueError(f"Target column '{target_col}' not found in dataset")
            
            # Statistical overview
            print("\nDataset Overview:")
            print(self.df.describe())
            
            # Prepare features and target
            self.X = self.df.drop(columns=[target_col])
            self.y = self.df[target_col]
            self.feature_names = self.X.columns.tolist()
            self.target_name = target_col
            
            print(f"\nFeatures ({len(self.feature_names)}): {self.feature_names}")
            print(f"Target: {self.target_name}")
            
        except Exception as e:
            messagebox.showerror("Data Loading Error", f"Failed to load data: {str(e)}")
            raise
    
    def train_model(self):
        try:
            # Use specified hyperparameters
            hyperparameters = {
                'colsample_bytree': 1.0,
                'learning_rate': 0.1,
                'max_depth': 5,
                'n_estimators': 200,
                'subsample': 0.8,
                'random_state': 42
            }
            
            print("\nTraining model with hyperparameters:")
            for key, value in hyperparameters.items():
                print(f"  {key}: {value}")
            
            self.model = xgb.XGBRegressor(**hyperparameters)
            self.model.fit(self.X, self.y)
            
            # Calculate feature importance
            self.feature_importance = pd.DataFrame({
                'feature': self.feature_names,
                'importance': self.model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            print("\nFeature Importance:")
            print(self.feature_importance.head(10))
            
            # Calculate model performance metrics
            y_pred = self.model.predict(self.X)
            mse = np.mean((self.y - y_pred) ** 2)
            r2 = 1 - (np.sum((self.y - y_pred) ** 2) / np.sum((self.y - np.mean(self.y)) ** 2))
            
            print(f"\nModel Performance:")
            print(f"  R² Score: {r2:.4f}")
            print(f"  MSE: {mse:.4f}")
            
        except Exception as e:
            messagebox.showerror("Model Training Error", f"Failed to train model: {str(e)}")
            raise
    
    def create_gui(self):
        # Main window setup
        self.root = tk.Tk()
        self.root.title(f"Advanced {self.target_name} Predictor")
        self.root.geometry("1300x850")
        self.root.configure(bg=self.colors['background'])
        
        # Create notebook for tabs
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Create tabs
        self.prediction_tab = self.create_prediction_tab()
        self.analysis_tab = self.create_analysis_tab()
        self.history_tab = self.create_history_tab()
        self.model_info_tab = self.create_model_info_tab()
        
        # Add tabs to notebook
        self.notebook.add(self.prediction_tab, text="🔮 Prediction")
        self.notebook.add(self.analysis_tab, text="📊 Data Analysis")
        self.notebook.add(self.history_tab, text="📋 Prediction History")
        self.notebook.add(self.model_info_tab, text="ℹ️ Model Info")
        
        # Initialize prediction history
        self.prediction_history = []
        self.history_file = "prediction_history.json"
        self.load_history()
    
    def create_prediction_tab(self):
        # Prediction tab
        tab = ttk.Frame(self.notebook)
        
        # Main frame with two columns
        main_frame = ttk.Frame(tab)
        main_frame.pack(fill='both', expand=True, padx=15, pady=15)
        
        # Left column for inputs
        left_frame = ttk.Frame(main_frame, width=700)
        left_frame.pack(side='left', fill='both', expand=True, padx=(0, 10))
        
        # Right column for results
        right_frame = ttk.Frame(main_frame, width=500)
        right_frame.pack(side='right', fill='both', expand=False, padx=(10, 0))
        
        # Title
        title_label = tk.Label(left_frame, text=f"{self.target_name} Prediction", 
                              font=("Arial", 22, "bold"), 
                              bg=self.colors['background'],
                              fg=self.colors['primary'])
        title_label.pack(pady=(0, 15), anchor='w')
        
        # Input frame - Using tk.Frame instead of ttk.LabelFrame with font
        input_frame_container = tk.Frame(left_frame, bg=self.colors['primary'], relief='solid', borderwidth=2)
        input_frame_container.pack(fill='both', expand=True, pady=(0, 15))
        
        # Input frame title
        input_title = tk.Label(input_frame_container, text="📝 Input Parameters",
                              font=("Arial", 12, "bold"),
                              bg=self.colors['primary'],
                              fg='white',
                              pady=5)
        input_title.pack(fill='x')
        
        # Actual input frame
        input_frame = tk.Frame(input_frame_container, bg=self.colors['card_bg'], padx=15, pady=15)
        input_frame.pack(fill='both', expand=True)
        
        # Create input fields in two columns
        self.entries = {}
        num_features = len(self.feature_names)
        mid_point = (num_features + 1) // 2
        
        for i, feature in enumerate(self.feature_names):
            row = i % mid_point
            col = i // mid_point
            
            frame = tk.Frame(input_frame, bg=self.colors['card_bg'])
            frame.grid(row=row, column=col, padx=12, pady=6, sticky='w')
            
            # Label with feature importance indicator
            importance = self.feature_importance[
                self.feature_importance['feature'] == feature
            ]['importance'].values[0]
            
            color = self.get_importance_color(importance)
            
            label = tk.Label(frame, text=f"{feature}:", 
                           font=("Arial", 11, "bold"),
                           bg=self.colors['card_bg'],
                           fg=color)
            label.pack(side='left')
            
            # Entry field
            entry = ttk.Entry(frame, width=22, font=("Arial", 11))
            entry.pack(side='left', padx=(8, 0))
            self.entries[feature] = entry
            
            # Add min/max values as tooltip
            feature_min = self.X[feature].min()
            feature_max = self.X[feature].max()
            feature_mean = self.X[feature].mean()
            tooltip_text = f"Range: [{feature_min:.2f}, {feature_max:.2f}]\nMean: {feature_mean:.2f}\nImportance: {importance:.3f}"
            self.create_tooltip(label, tooltip_text)
        
        # Button frame
        button_frame = tk.Frame(left_frame, bg=self.colors['background'])
        button_frame.pack(pady=10)
        
        # Predict button with loading indicator
        self.predict_btn = tk.Button(button_frame, text=f"🔮 Predict {self.target_name}", 
                                     font=("Arial", 12, "bold"),
                                     bg=self.colors['primary'],
                                     fg='white',
                                     padx=20,
                                     pady=8,
                                     relief='raised',
                                     command=self.threaded_predict)
        self.predict_btn.pack(side='left', padx=5)
        
        # Clear button
        clear_btn = tk.Button(button_frame, text="🗑️ Clear All", 
                              font=("Arial", 11),
                              bg=self.colors['warning'],
                              fg='white',
                              padx=15,
                              pady=5,
                              command=self.clear_inputs)
        clear_btn.pack(side='left', padx=5)
        
        # Auto-fill average values button
        avg_btn = tk.Button(button_frame, text="📊 Fill with Average Values",
                           font=("Arial", 11),
                           bg=self.colors['info'],
                           fg='white',
                           padx=15,
                           pady=5,
                           command=self.fill_average_values)
        avg_btn.pack(side='left', padx=5)
        
        # ====== RIGHT COLUMN - RESULTS ======
        
        # Result container with border
        result_container = tk.Frame(right_frame, bg=self.colors['primary'], 
                                   relief='raised', borderwidth=2)
        result_container.pack(fill='both', expand=True, pady=(20, 0))
        
        # Result header
        result_header = tk.Label(result_container, 
                                text="📈 PREDICTION RESULT",
                                font=("Arial", 14, "bold"),
                                bg=self.colors['primary'],
                                fg='white',
                                pady=10)
        result_header.pack(fill='x')
        
        # Main result frame
        self.result_main_frame = tk.Frame(result_container, 
                                         bg=self.colors['card_bg'],
                                         padx=20,
                                         pady=20)
        self.result_main_frame.pack(fill='both', expand=True, padx=2, pady=(0, 2))
        
        # Initial result display
        self.result_var = tk.StringVar(value="Awaiting prediction...")
        self.result_label = tk.Label(self.result_main_frame, 
                                    textvariable=self.result_var,
                                    font=("Arial", 26, "bold"),
                                    bg=self.colors['card_bg'],
                                    fg=self.colors['dark'])
        self.result_label.pack(pady=15)
        
        # Unit label
        self.unit_label = tk.Label(self.result_main_frame,
                                  text="Megapascals (MPa)",
                                  font=("Arial", 12),
                                  bg=self.colors['card_bg'],
                                  fg=self.colors['secondary'])
        self.unit_label.pack(pady=(0, 20))
        
        # Confidence indicator frame
        self.confidence_frame = tk.Frame(self.result_main_frame, 
                                        bg=self.colors['card_bg'])
        self.confidence_frame.pack(fill='x', pady=5)
        
        # Confidence label with icon
        confidence_icon = tk.Label(self.confidence_frame,
                                  text="🎯",
                                  font=("Arial", 12),
                                  bg=self.colors['card_bg'])
        confidence_icon.pack(side='left', padx=(0, 5))
        
        self.confidence_var = tk.StringVar(value="Confidence: --%")
        self.confidence_label = tk.Label(self.confidence_frame,
                                        textvariable=self.confidence_var,
                                        font=("Arial", 11, "bold"),
                                        bg=self.colors['card_bg'],
                                        fg=self.colors['primary'])
        self.confidence_label.pack(side='left')
        
        # Range indicator frame
        range_frame = tk.Frame(self.result_main_frame,
                              bg=self.colors['card_bg'])
        range_frame.pack(fill='x', pady=10)
        
        # Range label with icon
        range_icon = tk.Label(range_frame,
                             text="📊",
                             font=("Arial", 12),
                             bg=self.colors['card_bg'])
        range_icon.pack(side='left', padx=(0, 5))
        
        self.range_var = tk.StringVar(value="Dataset Range: -- to -- MPa")
        range_label = tk.Label(range_frame,
                              textvariable=self.range_var,
                              font=("Arial", 10),
                              bg=self.colors['card_bg'],
                              fg=self.colors['accent'])
        range_label.pack(side='left')
        
        # Status frame
        self.status_frame = tk.Frame(self.result_main_frame,
                                    bg=self.colors['card_bg'])
        self.status_frame.pack(fill='x', pady=(15, 5))
        
        self.status_var = tk.StringVar(value="Status: Ready")
        status_label = tk.Label(self.status_frame,
                               textvariable=self.status_var,
                               font=("Arial", 10, "italic"),
                               bg=self.colors['card_bg'],
                               fg=self.colors['dark'])
        status_label.pack()
        
        # Quality indicator (will be updated with prediction)
        self.quality_frame = tk.Frame(self.result_main_frame,
                                     bg=self.colors['card_bg'])
        self.quality_label = tk.Label(self.quality_frame,
                                     text="",
                                     font=("Arial", 10),
                                     bg=self.colors['card_bg'])
        self.quality_label.pack()
        
        # Time stamp
        self.timestamp_var = tk.StringVar(value="Last prediction: --")
        timestamp_label = tk.Label(self.result_main_frame,
                                  textvariable=self.timestamp_var,
                                  font=("Arial", 9),
                                  bg=self.colors['card_bg'],
                                  fg=self.colors['dark'])
        timestamp_label.pack(pady=(10, 0))
        
        return tab
    
    def create_analysis_tab(self):
        tab = ttk.Frame(self.notebook)
        
        # Analysis controls
        control_frame = tk.Frame(tab, bg=self.colors['background'])
        control_frame.pack(fill='x', padx=20, pady=10)
        
        buttons = [
            ("📊 Show Feature Importance", self.plot_feature_importance),
            ("📈 Show Data Distribution", self.plot_data_distribution),
            ("📉 Partial Dependence Plot", self.plot_partial_dependence),
            ("🌡️ Show Correlation Heatmap", self.plot_correlation_heatmap)
        ]
        
        for text, command in buttons:
            btn = tk.Button(control_frame, text=text,
                          font=("Arial", 10),
                          bg=self.colors['primary'],
                          fg='white',
                          padx=10,
                          pady=5,
                          command=command)
            btn.pack(side='left', padx=5)
        
        # Plot area
        self.analysis_frame = ttk.Frame(tab)
        self.analysis_frame.pack(fill='both', expand=True, padx=20, pady=10)
        
        return tab
    
    def create_history_tab(self):
        tab = ttk.Frame(self.notebook)
        
        # History controls
        control_frame = tk.Frame(tab, bg=self.colors['background'])
        control_frame.pack(fill='x', padx=20, pady=10)
        
        clear_btn = tk.Button(control_frame, text="🗑️ Clear History",
                             font=("Arial", 10),
                             bg=self.colors['danger'],
                             fg='white',
                             padx=10,
                             pady=5,
                             command=self.clear_history)
        clear_btn.pack(side='left', padx=5)
        
        export_btn = tk.Button(control_frame, text="📤 Export to CSV",
                              font=("Arial", 10),
                              bg=self.colors['success'],
                              fg='white',
                              padx=10,
                              pady=5,
                              command=self.export_history)
        export_btn.pack(side='left', padx=5)
        
        # History table with better styling
        columns = ("Timestamp", "Strength (MPa)", *[f[:15] for f in self.feature_names[:2]] + ["..."])
        
        # Create treeview with better styling
        style = ttk.Style()
        style.configure("Treeview.Heading", font=('Arial', 10, 'bold'))
        style.configure("Treeview", font=('Arial', 9), rowheight=25)
        
        self.history_tree = ttk.Treeview(tab, columns=columns, show="headings", height=15)
        
        # Configure columns
        col_widths = [150, 120, 100, 100, 80]
        for i, col in enumerate(columns):
            self.history_tree.heading(col, text=col)
            self.history_tree.column(col, width=col_widths[i], anchor='center')
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(tab, orient="vertical", command=self.history_tree.yview)
        self.history_tree.configure(yscrollcommand=scrollbar.set)
        
        self.history_tree.pack(side='left', fill='both', expand=True, padx=20, pady=10)
        scrollbar.pack(side='right', fill='y', pady=10)
        
        return tab
    
    def create_model_info_tab(self):
        tab = ttk.Frame(self.notebook)
        
        # Info frame
        info_frame = tk.Frame(tab, bg=self.colors['card_bg'], padx=20, pady=20)
        info_frame.pack(fill='both', expand=True)
        
        # Model information
        info_text = f"""
        📋 Model Information:
        
        🎯 Algorithm: XGBoost Regressor
        🎯 Target Variable: {self.target_name}
        
        ⚙️ Hyperparameters:
          • n_estimators: 200
          • max_depth: 5
          • learning_rate: 0.1
          • subsample: 0.8
          • colsample_bytree: 1.0
          • random_state: 42
        
        📊 Dataset Information:
          • Number of samples: {len(self.df):,}
          • Number of features: {len(self.feature_names)}
        
        📈 Model Performance:
          • R² Score: {self.calculate_r2():.4f}
          • MSE: {self.calculate_mse():.4f}
        
        🏆 Feature Importance (Top 5):
        """
        
        # Add top 5 features
        for i, row in self.feature_importance.head(5).iterrows():
            info_text += f"  {i+1}. {row['feature']}: {row['importance']:.4f}\n"
        
        # Data statistics
        info_text += f"""
        📊 Target Statistics:
          • Minimum: {self.y.min():.2f} MPa
          • Maximum: {self.y.max():.2f} MPa
          • Mean: {self.y.mean():.2f} MPa
          • Standard Deviation: {self.y.std():.2f} MPa
        """
        
        info_label = tk.Label(info_frame, text=info_text, 
                             font=("Courier", 10), 
                             justify="left",
                             bg=self.colors['card_bg'],
                             anchor="w")
        info_label.pack(fill='both', expand=True)
        
        return tab
    
    def calculate_r2(self):
        y_pred = self.model.predict(self.X)
        return 1 - (np.sum((self.y - y_pred) ** 2) / np.sum((self.y - np.mean(self.y)) ** 2))
    
    def calculate_mse(self):
        y_pred = self.model.predict(self.X)
        return np.mean((self.y - y_pred) ** 2)
    
    def get_importance_color(self, importance):
        max_importance = self.feature_importance['importance'].max()
        normalized = importance / max_importance
        
        if normalized > 0.5:
            return self.colors['danger']  # High importance - red
        elif normalized > 0.2:
            return self.colors['warning']  # Medium importance - orange
        else:
            return self.colors['primary']  # Low importance - blue
    
    def create_tooltip(self, widget, text):
        def on_enter(event):
            tooltip = tk.Toplevel()
            tooltip.wm_overrideredirect(True)
            tooltip.wm_geometry(f"+{event.x_root+10}+{event.y_root+10}")
            
            # Create a styled tooltip
            tooltip_frame = tk.Frame(tooltip, bg='lightyellow', relief='solid', borderwidth=1)
            tooltip_frame.pack()
            
            label = tk.Label(tooltip_frame, text=text, 
                           background="lightyellow", 
                           foreground='black',
                           font=("Arial", 9),
                           padx=5, pady=3)
            label.pack()
            widget.tooltip = tooltip
        
        def on_leave(event):
            if hasattr(widget, 'tooltip'):
                widget.tooltip.destroy()
        
        widget.bind("<Enter>", on_enter)
        widget.bind("<Leave>", on_leave)
    
    def fill_average_values(self):
        """Fill input fields with average values from training data"""
        for feature in self.feature_names:
            if feature in self.entries:
                avg_value = self.X[feature].mean()
                self.entries[feature].delete(0, tk.END)
                self.entries[feature].insert(0, f"{avg_value:.2f}")
    
    def threaded_predict(self):
        """Run prediction in separate thread to prevent GUI freezing"""
        self.predict_btn.config(state='disabled', text="⏳ Predicting...", bg=self.colors['warning'])
        self.status_var.set("Status: Predicting...")
        thread = threading.Thread(target=self.predict_target)
        thread.daemon = True
        thread.start()
    
    def predict_target(self):
        try:
            # Get input values
            inputs = {}
            for feature, entry in self.entries.items():
                value = entry.get().strip()
                if not value:
                    self.root.after(0, lambda: messagebox.showerror(
                        "Input Error", f"Please enter value for {feature}"))
                    self.root.after(0, self.enable_predict_button)
                    return
                
                try:
                    inputs[feature] = float(value)
                except ValueError:
                    self.root.after(0, lambda: messagebox.showerror(
                        "Input Error", f"Invalid number for {feature}"))
                    self.root.after(0, self.enable_predict_button)
                    return
            
            # Create input data
            input_data = pd.DataFrame([inputs], columns=self.feature_names)
            
            # Make prediction
            prediction = self.model.predict(input_data)[0]
            
            # Calculate confidence based on feature ranges
            confidence = self.calculate_confidence(inputs)
            
            # Update GUI in main thread
            self.root.after(0, lambda: self.display_result(prediction, confidence, inputs))
            
        except Exception as e:
            self.root.after(0, lambda: messagebox.showerror(
                "Prediction Error", f"Prediction failed: {str(e)}"))
            self.root.after(0, self.enable_predict_button)
    
    def calculate_confidence(self, inputs):
        """Calculate prediction confidence based on input values"""
        confidence = 92  # Base confidence
        
        # Adjust confidence based on whether values are within standard deviations of mean
        for feature, value in inputs.items():
            mean_val = self.X[feature].mean()
            std_val = self.X[feature].std()
            
            if abs(value - mean_val) > 2 * std_val:
                confidence -= 8  # Reduce confidence for outliers
            elif abs(value - mean_val) > 1.5 * std_val:
                confidence -= 4  # Slightly reduce confidence
            elif abs(value - mean_val) < 0.5 * std_val:
                confidence += 3  # Increase confidence for typical values
        
        # Ensure confidence is within reasonable bounds
        confidence = max(70, min(99, confidence))
        return confidence
    
    def enable_predict_button(self):
        self.predict_btn.config(state='normal', 
                               text=f"🔮 Predict {self.target_name}", 
                               bg=self.colors['primary'])
        self.status_var.set("Status: Ready")
    
    def display_result(self, prediction, confidence, inputs):
        # Update timestamp
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.timestamp_var.set(f"Last prediction: {current_time}")
        
        # Format result with color coding
        result_text = f"{prediction:.2f}"
        self.result_var.set(result_text)
        
        # Set result color based on value
        y_mean = self.y.mean()
        y_std = self.y.std()
        
        if prediction > y_mean + y_std:
            result_color = self.colors['danger']  # High - red
            quality = "High Strength"
        elif prediction < y_mean - y_std:
            result_color = self.colors['warning']  # Low - orange
            quality = "Low Strength"
        else:
            result_color = self.colors['success']  # Normal - green
            quality = "Normal Strength"
        
        self.result_label.config(fg=result_color)
        
        # Update confidence with color coding
        if confidence >= 90:
            conf_color = self.colors['success']
        elif confidence >= 80:
            conf_color = self.colors['warning']
        else:
            conf_color = self.colors['danger']
        
        self.confidence_var.set(f"Confidence: {confidence:.1f}%")
        self.confidence_label.config(fg=conf_color)
        
        # Show range information
        y_min, y_max = self.y.min(), self.y.max()
        self.range_var.set(f"Dataset Range: {y_min:.2f} to {y_max:.2f} MPa")
        
        # Update quality indicator
        self.quality_label.config(text=f"Quality: {quality}", fg=result_color)
        self.quality_frame.pack(pady=5)
        
        # Show comparison to mean
        diff = prediction - y_mean
        diff_text = f"Difference from mean: {diff:+.2f} MPa ({diff/y_std:+.1f}σ)"
        if hasattr(self, 'diff_label'):
            self.diff_label.config(text=diff_text)
        else:
            self.diff_label = tk.Label(self.result_main_frame,
                                      text=diff_text,
                                      font=("Arial", 10),
                                      bg=self.colors['card_bg'],
                                      fg=self.colors['dark'])
            self.diff_label.pack(pady=2)
        
        # Add percentile information
        percentile = np.sum(self.y < prediction) / len(self.y) * 100
        percentile_text = f"Percentile: {percentile:.1f}%"
        if hasattr(self, 'percentile_label'):
            self.percentile_label.config(text=percentile_text)
        else:
            self.percentile_label = tk.Label(self.result_main_frame,
                                            text=percentile_text,
                                            font=("Arial", 10),
                                            bg=self.colors['card_bg'],
                                            fg=self.colors['primary'])
            self.percentile_label.pack(pady=2)
        
        # Save to history
        self.save_to_history(prediction, inputs)
        
        # Visual feedback
        self.show_success_animation()
        
        # Update status
        self.status_var.set("Status: Prediction Complete ✓")
        
        self.enable_predict_button()
    
    def show_success_animation(self):
        # Simple visual feedback
        original_bg = self.result_main_frame.cget('bg')
        for i in range(3):
            self.root.after(i * 200, lambda: 
                self.result_main_frame.config(bg=self.colors['success']))
            self.root.after(i * 200 + 100, lambda: 
                self.result_main_frame.config(bg=original_bg))
    
    def save_to_history(self, prediction, inputs):
        history_entry = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'prediction': prediction,
            **inputs
        }
        
        self.prediction_history.append(history_entry)
        self.update_history_display()
        self.save_history()
    
    def update_history_display(self):
        # Clear existing items
        for item in self.history_tree.get_children():
            self.history_tree.delete(item)
        
        # Add recent entries with alternating colors
        for i, entry in enumerate(self.prediction_history[-50:]):
            values = [
                entry['timestamp'],
                f"{entry['prediction']:.2f}",
                *[str(round(entry[feature], 2)) for feature in self.feature_names[:2]],
                "..."
            ]
            item = self.history_tree.insert("", "end", values=values)
            
            # Alternate row colors for better readability
            if i % 2 == 0:
                self.history_tree.item(item, tags=('evenrow',))
    
    def save_history(self):
        try:
            with open(self.history_file, 'w') as f:
                json.dump(self.prediction_history, f, indent=2)
        except Exception as e:
            print(f"Failed to save history: {e}")
    
    def load_history(self):
        try:
            if os.path.exists(self.history_file):
                with open(self.history_file, 'r') as f:
                    self.prediction_history = json.load(f)
                self.update_history_display()
        except Exception as e:
            print(f"Failed to load history: {e}")
    
    def clear_inputs(self):
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.result_var.set("Awaiting prediction...")
        self.result_label.config(fg=self.colors['dark'])
        self.confidence_var.set("Confidence: --%")
        self.range_var.set("Dataset Range: -- to -- MPa")
        self.status_var.set("Status: Ready")
        self.timestamp_var.set("Last prediction: --")
        
        # Hide extra labels
        if hasattr(self, 'diff_label'):
            self.diff_label.pack_forget()
        if hasattr(self, 'percentile_label'):
            self.percentile_label.pack_forget()
        self.quality_frame.pack_forget()
    
    def clear_history(self):
        if messagebox.askyesno("Confirm", "Are you sure you want to clear all prediction history?"):
            self.prediction_history = []
            self.update_history_display()
            self.save_history()
    
    def export_history(self):
        try:
            df = pd.DataFrame(self.prediction_history)
            filename = f"{self.target_name.replace(' ', '_').replace('(', '').replace(')', '')}_predictions_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df.to_csv(filename, index=False)
            messagebox.showinfo("Export Successful", f"✅ History exported to {filename}")
        except Exception as e:
            messagebox.showerror("Export Failed", f"Failed to export: {str(e)}")
    
    def plot_feature_importance(self):
        self.clear_analysis_frame()
        
        # UPDATED: Set DPI to 500 for high-resolution output
        fig = Figure(figsize=(10, 6), dpi=500, facecolor=self.colors['background'])
        ax = fig.add_subplot(111, facecolor='white')
        
        # Plot feature importance
        importance_df = self.feature_importance.head(15)
        y_pos = np.arange(len(importance_df))
        
        colors = [self.get_importance_color(imp) for imp in importance_df['importance']]
        bars = ax.barh(y_pos, importance_df['importance'], color=colors, edgecolor='black')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(importance_df['feature'])
        ax.set_xlabel('Feature Importance Score', fontweight='bold')
        ax.set_title(f'Top 15 Most Important Features for {self.target_name}', 
                    fontsize=14, fontweight='bold', pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        
        # Add value labels on bars
        for bar, value in zip(bars, importance_df['importance']):
            ax.text(value, bar.get_y() + bar.get_height()/2, 
                   f' {value:.3f}', va='center', fontweight='bold')
        
        fig.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
    
    def plot_data_distribution(self):
        self.clear_analysis_frame()
        
        # UPDATED: Set DPI to 500 for high-resolution output
        fig = Figure(figsize=(10, 6), dpi=500, facecolor=self.colors['background'])
        ax = fig.add_subplot(111, facecolor='white')
        
        # Plot target distribution
        ax.hist(self.y, bins=30, alpha=0.7, color=self.colors['secondary'], 
                edgecolor='black', density=True)
        
        # Add normal distribution curve
        from scipy.stats import norm
        mu, std = norm.fit(self.y)
        xmin, xmax = ax.get_xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mu, std)
        ax.plot(x, p, 'k', linewidth=2, label=f'Normal fit: μ={mu:.2f}, σ={std:.2f}')
        
        ax.set_xlabel(self.target_name, fontweight='bold')
        ax.set_ylabel('Density', fontweight='bold')
        ax.set_title(f'Distribution of {self.target_name}', 
                    fontsize=14, fontweight='bold', pad=20)
        ax.grid(True, alpha=0.3)
        ax.legend()
        
        fig.tight_layout()
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
    
    def plot_partial_dependence(self):
        self.clear_analysis_frame()
        
        # UPDATED: Set DPI to 500 for high-resolution output
        fig = Figure(figsize=(12, 8), dpi=500, facecolor=self.colors['background'])
        
        # Plot PDP for top 4 features
        top_features = self.feature_importance.head(4)['feature'].tolist()
        
        for i, feature in enumerate(top_features, 1):
            ax = fig.add_subplot(2, 2, i, facecolor='white')
            try:
                pdp = partial_dependence(self.model, self.X, [feature])
                ax.plot(pdp['values'][0], pdp['average'][0], 
                       color=self.colors['accent'], linewidth=2, marker='o', markersize=4)
                ax.set_xlabel(feature, fontweight='bold')
                ax.set_ylabel('Partial Dependence', fontweight='bold')
                ax.set_title(f'PDP for {feature}', fontweight='bold')
                ax.grid(True, alpha=0.3)
            except Exception as e:
                ax.text(0.5, 0.5, f"Error plotting {feature}", 
                       ha='center', va='center', transform=ax.transAxes)
        
        fig.suptitle(f'Partial Dependence Plots for {self.target_name}', 
                    fontsize=16, fontweight='bold', y=0.98)
        fig.tight_layout(rect=[0, 0, 1, 0.96])
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
    
    def plot_correlation_heatmap(self):
        self.clear_analysis_frame()
        
        # UPDATED: Set DPI to 500 for high-resolution output
        fig = Figure(figsize=(12, 10), dpi=500, facecolor=self.colors['background'])
        ax = fig.add_subplot(111, facecolor='white')
        
        # Calculate correlation matrix
        corr_matrix = pd.concat([self.X, self.y], axis=1).corr()
        
        # Plot heatmap
        im = ax.imshow(corr_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
        
        # Set labels
        ax.set_xticks(np.arange(len(corr_matrix.columns)))
        ax.set_yticks(np.arange(len(corr_matrix.columns)))
        ax.set_xticklabels(corr_matrix.columns, rotation=90, fontsize=8)
        ax.set_yticklabels(corr_matrix.columns, fontsize=8)
        
        # Add correlation values in cells
        for i in range(len(corr_matrix.columns)):
            for j in range(len(corr_matrix.columns)):
                text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                             ha="center", va="center", color="black", fontsize=7)
        
        # Add colorbar
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel('Correlation Coefficient', rotation=-90, va="bottom", fontweight='bold')
        
        ax.set_title('Feature Correlation Heatmap', fontsize=14, fontweight='bold', pad=20)
        
        canvas = FigureCanvasTkAgg(fig, self.analysis_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill='both', expand=True)
    
    def clear_analysis_frame(self):
        for widget in self.analysis_frame.winfo_children():
            widget.destroy()
    
    def run(self):
        # Configure ttk styles
        style = ttk.Style()
        style.theme_use('clam')
        
        # Configure treeview colors
        style.configure("Treeview", 
                       background=self.colors['card_bg'],
                       fieldbackground=self.colors['card_bg'],
                       foreground=self.colors['dark'])
        style.map('Treeview', background=[('selected', self.colors['primary'])])
        
        self.root.mainloop()


def main():
    try:
        print("🚀 Starting Compressive Strength Predictor Application...")
        app = CompressiveStrengthPredictor()
        print("✅ Application initialized successfully")
        app.run()
    except Exception as e:
        print(f"❌ Application Error: {str(e)}")
        messagebox.showerror("Application Error", f"Failed to start application: {str(e)}")


if __name__ == "__main__":
    main()

🚀 Starting Compressive Strength Predictor Application...
📋 Original columns: 27
✅ Cleaned columns: 27
📐 Dataset shape: (450, 27)

Dataset Overview:
              OPC          FA          SF        GGBS          MK          LP  \
count  450.000000  450.000000  450.000000  450.000000  450.000000  450.000000   
mean    60.555556   21.111111    9.583333    2.916667    2.916667    2.916667   
std     13.543719   10.088080    6.529124    5.159618    5.159618    5.159618   
min     50.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%     50.000000   15.000000    5.000000    0.000000    0.000000    0.000000   
50%     50.000000   20.000000    8.750000    0.000000    0.000000    0.000000   
75%     70.000000   30.000000   15.000000    5.000000    5.000000    5.000000   
max    100.000000   50.000000   20.000000   15.000000   15.000000   15.000000   

       Specific  gravity  Blaines  fineness,  cm2/gm  \
count         450.000000                   450.000000   
mean    